# Extaer características audios libreria TORCH

Torch es una biblioteca que nos permite hacer uso de funciones que usaremos en este caso para extraer las siguientes características de los audios:

- Espectrograma del audio
- Amplitud en decibelios
- Nivel de volumen del audio
- MFCC
- PSD
- Centroide espectral del audio

Autor: Maider Murugarren Ilundain

Antes de empezar se instalan las librerias necesarias en este caso.

In [1]:
#! pip install vgg-pytorch
#! pip install torchaudio
#! pip install torchvision

In [16]:
# Importamos los paquetes necesarios para la ejecución completa.
import csv
import os
import torch
import torchaudio
import numpy as np
import pandas as pd

def extraer_caracteristicas_PyThorch(ruta, nombre_csv):
    '''
    El código consistira en un bucle for que irá recorriendo todos los audios disponibles para extraer 
    de cada uno las características de nuestro interes, almacenarlas en un diccionario. Y finalmente 
    generar un archivo ".csv" donde se almacenarán las características de cada audio de forma ordenada.
    
    Parámetros:
    - ruta: directorio de la localización de los audios a analizar.
    - nombre_csv: nombre con el que se almacenarán los datos en el csv.
    
    '''
    # Ruta de la carpeta que contiene los archivos de audio
    archivos_audio = os.listdir(ruta)

    # Crear un diccionario para almacenar las características de los archivos de audio
    caract_dict = {}

    # Recorrer cada archivo en la carpeta
    for archivo in archivos_audio:

        # Obtener el ID y el tipo del archivo
        tipo = archivo[-5]
        ID = archivo[:5]

        # Convertimos en arrays de dimensiones 1
        ID_array = np.array([ID])
        tipo_array = np.array([tipo])

        # Cargar el archivo 
        file_path = os.path.join(ruta, archivo)
        waveform, sample_rate = torchaudio.load(file_path)
        
        try:
            # Transformaciones a forma de onda
            spectrogram = torchaudio.transforms.Spectrogram()(waveform)
            amplitude_to_db = torchaudio.transforms.AmplitudeToDB()(spectrogram)
            loudness = torchaudio.transforms.Loudness(sample_rate=sample_rate)(waveform)
            mfcc = torchaudio.transforms.MFCC(sample_rate=sample_rate)(waveform)
            psd = torchaudio.transforms.Spectrogram(n_fft=2048)(waveform)
            spectral_centroid = torchaudio.transforms.SpectralCentroid(sample_rate=sample_rate)(waveform)

            # Calcular la media 
            amplitude_to_db_mean = torch.mean(amplitude_to_db)
            loudness_mean = torch.mean(loudness, axis=0)
            mfcc_mean = torch.mean(mfcc)
            psd_mean = torch.mean(psd)
            spectral_centroid_mean = torch.mean(spectral_centroid)
            spectrogram_mean = torch.mean(spectrogram)

            # Agregar las características al diccionario 
            caract_dict[archivo] = np.concatenate((ID_array, tipo_array, amplitude_to_db_mean.reshape(-1,1)[0], 
                                                    loudness_mean.reshape(-1,1)[0], 
                                                    mfcc_mean.reshape(-1,1)[0], 
                                                    psd_mean.reshape(-1,1)[0], 
                                                    spectral_centroid_mean.reshape(-1,1)[0], 
                                                    spectrogram_mean.reshape(-1,1)[0]), axis=0)
            
        except RuntimeError as e:
            print(f"Error al procesar el archivo {archivo}: {str(e)}")

    # Escribir el contenido del diccionario en un archivo CSV
    with open(nombre_csv, 'w', newline='') as archivo_csv:
        writer = csv.writer(archivo_csv, delimiter=';')

        # Escribir los nombres de las características en la primera fila del CSV
        writer.writerow(['archivo', 'ID', 'tipo', 'amplitude_to_db', 'loudness', 'mfcc', 'psd', 'spectral_centroid', 'spectrogram'])

        # Escribir las características en cada línea del CSV
        for archivo, caracteristicas in caract_dict.items():
            writer.writerow([archivo] + list(caracteristicas))


Llamaremos a la función en la carpeta donde se alojan cada tipo de audios y se guardarán en el mismo los csv correspondientes.

In [ ]:
# Función para llamar a cada subconjunto de datos de forma automática
def carac_torch(tipo):
    """
    Llamar a cada directorio para acceder a todos los audios, creando a la vez el nombre de los csv donde 
    se almacenan las características extraidas en cada caso..

    Parámetros:
    - tipo: El número para acceder a todos los tipos de audios.

    Retorna:
    - La extracción de las características por la función anterior.
    """
    
    # Indicamos la carpeta base donde están todos los audios.
    carpeta_base = "Tipos_audios"
    # Completamos las posibles rutas con el tipo de audios.
    if (tipo == '1'):
        carpeta_tipo = carpeta_base+"/vocal_A_1/"
    elif (tipo == '2'):
        carpeta_tipo = carpeta_base+"/vocal_I_2/"
    elif (tipo == '3'):
        carpeta_tipo = carpeta_base+"/vocal_U_3/"
    elif (tipo == '4'):
        carpeta_tipo = carpeta_base+"/palabra_campana_4/"
    elif (tipo == '5'):
        carpeta_tipo = carpeta_base+"/palabra_gato_5/"
    elif (tipo == '6'):
        carpeta_tipo = carpeta_base+"/palabra_petaca_6/"
    
    # Se crea el nombre del csv a almacenar
    nombre_csv = "caracteristicas_audios_Vgg-pythorch_"+tipo+".csv"
    
    return extraer_caracteristicas_Spicy(carpeta_tipo, nombre_csv)

In [ ]:
# Llamamos a todas las carpetas de los audios
carac_torch('1')
carac_torch('2')
carac_torch('3')
carac_torch('4')
carac_torch('5')
carac_torch('6')